In [1]:
# Importing required libraries
import sys                                                         # Import sys libraries
reload(sys)
sys.setdefaultencoding('utf-8')                                    # setting encoding to utf-8 for Python 2.7
import pandas as pd                               
import numpy as np
from __future__ import unicode_literals
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import TaggedLineDocument
from gensim.models import Doc2Vec
from pyemd import emd
from gensim.similarities import WmdSimilarity

#Importing the text data into Pandas DataFrame
text = pd.read_csv('/home/ankit/Downloads/Text_Similarity_Dataset.csv')

#Function to do some text preprocessing
def text_preprocessing(text):  
    words = word_tokenize(text)                                                   #Tokenizing data into words
    corpus = []
    for word in words:
        review_text = re.sub(r"[^A-Za-z0-9()-\%\$\xc2\xa3]", " ", word)        #Incorporating alphanumeric and some special characters
        #review_text = re.sub(r'(\d)\s+(\d)', r'\1\2', review_text)
        review_text = re.sub(r"\s+", " ", review_text)                            #Removing whitespaces and tabs
        review_text = re.sub(r" ", "", review_text)
        review_text.strip()
        corpus.append(review_text)
    clean_words = [word for word in corpus if word not in stopwords.words('english')]    #Removing the stop words
    return clean_words 

#Opening file to write similarity score
f= open("similarity_score_new1.csv","a+")
f.write("Unique_ID")                                   #Writing headings of output csv file
f.write(",")
f.write("Similarity_Score\n")
f.close()

#Converting text data into numeric feature and then performing WMD similarity calculation

labels = list(text['Unique_ID'].astype(str))           # Converting Unique_ID column text data into list of strings for Tagged document 
for i in range(text.shape[0]): 
    text1 = text_preprocessing(text['text1'][i])       # Preprosessing the text 1 column data
    text2 = text_preprocessing(text['text2'][i])       # Preprosessing the text 2 column data
    doc = []
    doc1 = TaggedDocument(text1, labels[i])            #Creating Tagged Document for text 1 column as input to doc2vec 
    doc2 = TaggedDocument(text2, labels[i])            #Creating Tagged Document for text 2 column as input to doc2vec
    doc.append(doc1)       
    doc.append(doc2)                                   #Appending text1 and text 2 data 
    
    #Initializing doc2vec word embedding model
    model = Doc2Vec(dm = 1, min_count=1, window=10, vector_size=200, sample=1e-4, negative=10, alpha=0.025, min_alpha=0.001) 
    
    model.build_vocab(doc)                                                #Building vocabulary with text data
    
    model.train(doc,total_examples=model.corpus_count,epochs=10)          #Training the model with text data and no of epochs is set to 10  
    model.init_sims(replace=True)                                         #Normalizing the vector to calculate similarity distance between 0 and 1
    
    f= open("similarity_score.csv","a+")
    f.write(str(i))
    f.write(",")
    f.write(str(model.wmdistance(text1, text2)))                          #Calculate the WMD distance and saving it in the file
    f.write("\n")
    f.close()

KeyboardInterrupt: 